In [1]:
#import dependencies

import pandas as pd
from sqlalchemy import create_engine

In [2]:
##EXTRACT
#apply csv file to parameter
csv_file = "imdb.csv"

#read in csv file, drop rows with corrupted data as identified when reviewing CSV
movie_df = pd.read_csv(csv_file, error_bad_lines=False,warn_bad_lines=False)
movie_df.head()

,fn,tid,title,wordsInTitle,url,imdbRating,ratingCount,duration,year,type,...,News,RealityTV,Romance,SciFi,Short,Sport,TalkShow,Thriller,War,Western
0,titles01/tt0012349,tt0012349,Der Vagabund und das Kind (1921),der vagabund und das kind,http://www.imdb.com/title/tt0012349/,8.4,40550.0,3240.0,1921.0,video.movie,...,0,0,0,0,0,0,0,0,0,0
1,titles01/tt0015864,tt0015864,Goldrausch (1925),goldrausch,http://www.imdb.com/title/tt0015864/,8.3,45319.0,5700.0,1925.0,video.movie,...,0,0,0,0,0,0,0,0,0,0
2,titles01/tt0017136,tt0017136,Metropolis (1927),metropolis,http://www.imdb.com/title/tt0017136/,8.4,81007.0,9180.0,1927.0,video.movie,...,0,0,0,1,0,0,0,0,0,0
3,titles01/tt0017925,tt0017925,Der General (1926),der general,http://www.imdb.com/title/tt0017925/,8.3,37521.0,6420.0,1926.0,video.movie,...,0,0,0,0,0,0,0,0,0,0
4,titles01/tt0021749,tt0021749,Lichter der Großstadt (1931),lichter der gro stadt,http://www.imdb.com/title/tt0021749/,8.7,70057.0,5220.0,1931.0,video.movie,...,0,0,1,0,0,0,0,0,0,0


In [4]:
##TRANSFORM
#Creating a clean dataframe with only pertinent columns
clean_movie_df = movie_df[['title','wordsInTitle','imdbRating','ratingCount','type']]
#Rename columns in dataframe to match up with table in SQL database
transformed_movie_df = clean_movie_df.rename(columns={"wordsInTitle":"words_in_title",
                                                      "imdbRating":"imdb_rating",
                                                      "ratingCount":"rating_count",
                                                      "type":"medium"})


transformed_movie_df.head()

,title,words_in_title,imdb_rating,rating_count,medium
0,Der Vagabund und das Kind (1921),der vagabund und das kind,8.4,40550.0,video.movie
1,Goldrausch (1925),goldrausch,8.3,45319.0,video.movie
2,Metropolis (1927),metropolis,8.4,81007.0,video.movie
3,Der General (1926),der general,8.3,37521.0,video.movie
4,Lichter der Großstadt (1931),lichter der gro stadt,8.7,70057.0,video.movie


In [13]:
final_movie_df = transformed_movie_df.loc[transformed_movie_df['medium']== 'video.movie']

final_movie_df.head()

,title,words_in_title,imdb_rating,rating_count,medium
0,Der Vagabund und das Kind (1921),der vagabund und das kind,8.4,40550.0,video.movie
1,Goldrausch (1925),goldrausch,8.3,45319.0,video.movie
2,Metropolis (1927),metropolis,8.4,81007.0,video.movie
3,Der General (1926),der general,8.3,37521.0,video.movie
4,Lichter der Großstadt (1931),lichter der gro stadt,8.7,70057.0,video.movie


In [15]:
##LOAD
#Create Connection to SQL DB
rds_connection_string = "postgres:Dr@gon88@localhost:5432/ETL_Grp_8"
engine = create_engine(f'postgresql://{rds_connection_string}')


In [16]:
#Veryifying Table Name
engine.table_names()

['movies']

In [17]:
#Load the dataframe into the SQL Database
transformed_movie_df.to_sql(name='movies', con=engine, if_exists='append', index=False)

In [18]:
#Verify Data in in the DB
pd.read_sql_query('select * from movies', con=engine).head()

,id,title,words_in_title,imdb_rating,rating_count,medium
0,1,Der Vagabund und das Kind (1921),der vagabund und das kind,8.4,40550.0,video.movie
1,2,Goldrausch (1925),goldrausch,8.3,45319.0,video.movie
2,3,Metropolis (1927),metropolis,8.4,81007.0,video.movie
3,4,Der General (1926),der general,8.3,37521.0,video.movie
4,5,Lichter der Großstadt (1931),lichter der gro stadt,8.7,70057.0,video.movie
